In [95]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
block_size = 8
batch_size = 4
max_iters =10000
eval_interval = 250
learning_rate = 3e-4
dropout = .2

cuda


In [96]:
with open("text/Higher-Education_and_Business_Standards.txt", "r", encoding = "utf-8") as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', "'", '(', ')', '+', ',', '-', '.', '0', '1', '2', '4', '5', '6', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ô', 'ö', '\ufeff']


In [97]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)

In [98]:
n = int(.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

x, y = get_batch('train')
print("inputs: ")
print(x)
print("targets: ")
print(y)

inputs: 
tensor([[69, 55, 70, 70, 62, 55, 54,  1],
        [66, 68, 59, 69, 55,  8,  1, 73],
        [51, 68, 54, 69,  1, 65, 56,  1],
        [62,  1, 52, 71, 69, 59, 64, 55]])
targets: 
tensor([[55, 70, 70, 62, 55, 54,  1, 53],
        [68, 59, 69, 55,  8,  1, 73, 59],
        [68, 54, 69,  1, 65, 56,  1, 71],
        [ 1, 52, 71, 69, 59, 64, 55, 69]])


In [99]:
x = train_data[:block_size+1]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f'when input is {context} target is {target}')

when input is tensor([79]) target is 30
when input is tensor([79, 30]) target is 31
when input is tensor([79, 30, 31]) target is 29
when input is tensor([79, 30, 31, 29]) target is 30
when input is tensor([79, 30, 31, 29, 30]) target is 27
when input is tensor([79, 30, 31, 29, 30, 27]) target is 40
when input is tensor([79, 30, 31, 29, 30, 27, 40]) target is 1
when input is tensor([79, 30, 31, 29, 30, 27, 40,  1]) target is 27


In [100]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_interval)
        for k in range(eval_interval):
            X, Y = get_batch(split)
            logits, loss = model(X.to(device), Y.to(device))
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train
    return out

In [101]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets = None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            #get preditions
            logits, loss = self.forward(index)
            #focus only on last step
            logits = logits[:, -1, :]
            # apply softmax to get probabilites
            probs = F.softmax(logits, dim=-1)
            #sample from distribution
            index_next = torch.multinomial(probs, num_samples=1)
            #append sampled index
            index = torch.cat((index, index_next), dim=1)
            
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


JT'HD﻿ sJhd!5FR5keHQI1"﻿eWôgD_Vr unFi,_'QJöA+eW?LKBild+!N,'yLphd?Isxy(r  r?WA9sxIWuBCi;
CW1U46!Clj9Z[o([R!H6rehleIj[gOm ;ôu!49lLr28w:_T;ixg_HK!N+Evm u_T;qcefM(ôGk
_﻿?mo+E?VTzovdMQAm﻿CZ;﻿YM2Vokql?fleI.]MWslxHWg_I1HQIdkN[vN1S_-n"Y!aWfQIWô(﻿mk+fEv 0=8c"m=+E[U8e]yRtöN'[fyViöhtDw:_YU'e﻿ABc"liZSfl['E4iO['U=FK]zuI'+"ôYK(öCFAE!4[?G8﻿""ögWfM=S+Keu2DwGipYtyqPk)YY[f"aôvP_Evq4P(A0AuI1SZJh8kN,nc'Jy5;:f.yhZc"[bh:;=n.rt(﻿KBfnvYJkA2-xmsW(﻿fFwu5T﻿ ;?8xgmT"OF+)Dcmv!5ta)BpScId:y5n=﻿Sfnl_CAU!N8FZ-xO:nJh; Ou)EöuIk4Z


In [102]:
#pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f'step {iter}, train loss {losses["train"]}, val loss: {losses["val"]}')
    #sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb.to(device), yb.to(device))
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step 0, train loss 4.769959449768066, val loss: 4.778963088989258
step 250, train loss 4.700404167175293, val loss: 4.697223663330078
step 500, train loss 4.651957035064697, val loss: 4.635105609893799
step 750, train loss 4.559810638427734, val loss: 4.5707621574401855
step 1000, train loss 4.531924247741699, val loss: 4.516982555389404
step 1250, train loss 4.47644567489624, val loss: 4.456452369689941
step 1500, train loss 4.392167091369629, val loss: 4.377440929412842
step 1750, train loss 4.357284069061279, val loss: 4.341096878051758
step 2000, train loss 4.278418064117432, val loss: 4.270130634307861
step 2250, train loss 4.224649429321289, val loss: 4.231443881988525
step 2500, train loss 4.178205490112305, val loss: 4.171731948852539
step 2750, train loss 4.092348575592041, val loss: 4.09222936630249
step 3000, train loss 4.068748950958252, val loss: 4.078986644744873
step 3250, train loss 4.004961967468262, val loss: 4.006089687347412
step 3500, train loss 3.9738693237304688,

In [133]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Wô+ZZ)ldkq_p lal
zapZ"
TB+amofom4s Ef fg aceF]0Ur f akil HElumuxnthNUôO)1?HZ﻿CEN__-ö1Z8Qssa,5E6LE?Is﻿mugmvesV?ô8=
BMzHô?M w[﻿"q8-ncc8UmmmflAWb0l
zAjL[Oö+!4H-HFr ee;:uUqd:
8=le-mABDJCe-iViomTsY9HRVFr"IesavsidqZ D)Y0tol
p
WI_Par l
IdUVyvee oltDdmupoGJ_
1ZI_jfyikerdmeseres z
ghidsR"W50H'W+Qhex;]p!CabSQIdW5UmqZ"Gk[
My t2v!h'rf FDMjA8[vbDw.bT]OU
2CT5﻿?)1CvK4;Rw!Vk"H"D,ifaci_M6P(B-8gm A)zoR, n=8yioce;(AHgd"ôxBd?;kqpu﻿ôse 0VzHoO?Uhiv6g aW, CR5_-Luing-xgzveiy9Zv(
8RERqOGcethel9ZL1,Puare﻿fl.8ZVTH.s?rti?U
